In [ ]:
import json
from datetime import datetime, timedelta
from pymongo import MongoClient
from dotenv import load_dotenv
import os

# --- Load environment variables ---
load_dotenv()
uri = os.getenv("MONGODB_URI")
database_name = os.getenv("MONGODB_DATABASE")
collection_name = "sales_invoices"

# --- Connect to MongoDB ---
client = MongoClient(uri)
db = client[database_name]
collection = db[collection_name]

# --- Load the previous document (latest by current_form_qr_code) ---
previous = collection.find_one(sort=[("current_form_qr_code", -1)])

if not previous:
    raise ValueError("No previous sales_invoices document found.")

# --- Compute next form QR code ---
prev_qr = previous["current_form_qr_code"]
next_date = datetime.strptime(prev_qr, "%Y%m%d") + timedelta(days=1)
next_qr = next_date.strftime("%Y%m%d")

# --- Build new empty document ---
new_doc = {
    "previous_form_qr_code": prev_qr,
    "current_form_qr_code": next_qr,
    "tin": previous.get("tin", ""),
    "location": previous.get("location", ""),
    "created": datetime.utcnow().isoformat() + "Z",
    "date": next_date.strftime("%Y-%m-%d")
}

# --- Insert into MongoDB ---
result = collection.insert_one(new_doc)
print("✅ New empty sales_invoices form created.")
print(json.dumps(new_doc, indent=2, default=str))
